In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

from settings import Settings
from dataloader_ml import DataHandler
import json
import matplotlib.pyplot as plt

# Load configuration using the Settings class
settings = Settings(
    default_config="default.toml",
    project_config="example.toml"
)

print(f"Project: {settings.get('general.name')}")
print(f"Base path: {settings.get('data.base_path')}")
print(f"Model folder: {settings.get('data.model_folder')}")

# Create necessary directories
settings.create_directories()

In [ ]:
data = DataHandler(settings)
data.train_test_split(test_ratio=0.1, valid_ratio=0.2)
distribution = data.run_data_loader()

with open(settings.get('data.model_folder')+"/distribution.json", 'w') as f:    
    json.dump(distribution, f, indent=4)

### Loading and splitting data

In [ ]:
plt.imshow(data.train_data[0].regions[0]['image'][0,:,:], cmap='gray')
plt.imshow(data.train_data[0].regions[0]['mask'][0,:,:], cmap='Blues',alpha=0.25)
plt.show()

### Training model
#### you will find the best model and its metrices in 'base_path'/model

In [ ]:
image, mask, weights = data.show_example_batch()

for index in range(0, image.shape[0]):
    plt.figure(figsize=(14, 3))
    plt.subplot(1, 4, 1)
    plt.title("Image")
    plt.imshow(image[index, 0, :], cmap='gray', vmin=-2, vmax=2)
    plt.subplot(1, 4, 2)
    plt.title("Mask")
    plt.imshow(mask[index, 0, :], cmap='gray')
    plt.subplot(1, 4, 3)
    plt.title("weight")
    plt.imshow(image[index, 0, :], cmap='gray', vmin=-2, vmax=2)
    plt.imshow(weights[index, 0, :], cmap='Reds', alpha=0.5)
    plt.subplot(1, 4, 4)
    plt.title("histogram")
    plt.hist(image[index, 0, :].flatten(), bins=100, range=(-2, 2))
    plt.show()
    plt.close()

In [ ]:
from segmagic_ml import Segmagic

seg = Segmagic(settings)
seg.train_model(
    data
)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(seg.model.tml.metrics['val_f1'].history['micro'])
plt.plot(seg.model.tml.metrics['train_f1'].history['micro'])

### Testing model
#### Test results are stored under 'base_path'/Testing

In [ ]:
from segmagic_ml import Segmagic

seg = Segmagic(settings)
################## chose image to predict ##################
seg.test_images(data)
#seg.predict_image(image_to_predict, labels,show=True)